### LOAD THE DATA

In [1]:
import numpy as np
import pandas as pd

table = pd.read_csv("positionDataIncorrect.csv", low_memory=False)
display(table.head())
print(len(table))

,ENEMY,ROWID,CH1Id,CH1keyRune,CH1spell1,CH1spell2,CH2Id,CH2JNGkeyRune,CH2spell1,CH2spell2,...,CH3spell1,CH3spell2,CH4champId,CH4keyRune,CH4spell1,CH4spell2,CH5champId,CH5keyRune,CH5spell1,CH5spell2
0,0,3,25,4,4,9,26,11,4,0,...,4,5,28,13,4,0,23,4,4,0
1,0,4,26,11,4,7,22,13,4,0,...,4,2,118,1,4,9,62,14,4,5
2,0,8,68,8,4,5,121,3,4,7,...,4,5,32,13,4,5,54,4,4,9
3,0,12,29,4,4,9,102,3,4,7,...,4,5,137,8,4,2,81,2,4,0
4,0,16,71,2,4,7,111,15,4,5,...,4,5,60,4,4,5,129,0,4,9


178095


### Prepare the data extracting x

In [2]:
import time
import torch

b = time.time()
batches = 100
bs = len(table)//batches

array = table.values[:len(table)]
i = np.arange(len(table))

misc = array[np.ix_(i,range(2))]

x = array[np.ix_(i,range(2, 22))]

ext = bs-(len(x)%bs)
if ext > 0:
    batches += 1

x = np.concatenate((x, x[:ext]))

x = torch.from_numpy(x)

print("Execution time:", round(time.time()-b, 2), "seconds")

Execution time: 0.05 seconds


### COPY THE MODEL

In [3]:
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data

device = torch.device("cuda:0")

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        
        self.i = torch.arange(5).view(1, 5).to(device)
        
        self.id = torch.zeros(5, bs, 141).to(device)
        self.rune = torch.zeros(5, bs, 17).to(device)
        self.spells = torch.zeros(5, bs, 10).to(device)
        
        self.idC = nn.Linear(141, 70) #50 best
        self.runeC = nn.Linear(17, 14) #9 best
        self.spellsC = nn.Linear(10, 14)#10 best
        
        self.c = nn.Linear(70+14+14, 100) #40 best
        
        self.fc2 = nn.Linear((100)*5, 450) #40, 200 best
        self.fc3 = nn.Linear(450, 25) #200 best
        #self.fc4 = nn.Linear(400, 25)

    def forward(self, x):
        
        self.id.zero_()
        self.rune.zero_()
        self.spells.zero_()
        
        k1 = x[:, self.i*4].long().transpose(0, 2).view(5, bs, 1)
        k2 = x[:, self.i*4+1].long().transpose(0, 2).view(5, bs, 1)
        k3 = x[:, self.i*4+2].long().transpose(0, 2).view(5, bs, 1)
        k4 = x[:, self.i*4+3].long().transpose(0, 2).view(5, bs, 1)
        
        self.id.scatter_(2, k1, 1)
        self.rune.scatter_(2, k2, 1)
        self.spells.scatter_(2, k3, 1)
        self.spells.scatter_(2, k4, 1)
        
        x = torch.cat((
            F.relu(self.c(torch.cat((
                F.relu(self.idC(self.id[0])), 
                F.relu(self.runeC(self.rune[0])), 
                F.relu(self.spellsC(self.spells[0]))
                ), dim=1))),
            F.relu(self.c(torch.cat((
                F.relu(self.idC(self.id[1])), 
                F.relu(self.runeC(self.rune[1])), 
                F.relu(self.spellsC(self.spells[1]))
            ), dim=1))),
            F.relu(self.c(torch.cat((
                F.relu(self.idC(self.id[2])), 
                F.relu(self.runeC(self.rune[2])), 
                F.relu(self.spellsC(self.spells[2]))
            ), dim=1))),
            F.relu(self.c(torch.cat((
                F.relu(self.idC(self.id[3])), 
                F.relu(self.runeC(self.rune[3])), 
                F.relu(self.spellsC(self.spells[3]))
            ), dim=1))),
            F.relu(self.c(torch.cat((
                F.relu(self.idC(self.id[4])), 
                F.relu(self.runeC(self.rune[4])), 
                F.relu(self.spellsC(self.spells[4]))
            ), dim=1)))
        ), dim=1)
        
        x = F.relu(self.fc2(x))
        x = F.sigmoid(self.fc3(x))

        return x
    
net = Net()
net.to(device)
net.train(False)

D:\ANACONDA\lib\site-packages\torch\cuda\__init__.py:116: UserWarning: 
    Found GPU0 GeForce GTX 770 which is of cuda capability 3.0.
    PyTorch no longer supports this GPU because it is too old.
    
  warnings.warn(old_gpu_warn % (d, name, major, capability[1]))


Net(
  (idC): Linear(in_features=141, out_features=70, bias=True)
  (runeC): Linear(in_features=17, out_features=14, bias=True)
  (spellsC): Linear(in_features=10, out_features=14, bias=True)
  (c): Linear(in_features=98, out_features=100, bias=True)
  (fc2): Linear(in_features=500, out_features=450, bias=True)
  (fc3): Linear(in_features=450, out_features=25, bias=True)
)

### IMPORT THE PARAMETERS

In [4]:
net.load_state_dict(torch.load('92.55.pth'))

### Classify the teams

In [5]:
y = torch.tensor([]).to(device)

with torch.no_grad():
    # process in big batches
    for i in range(batches):
        inputs = x[bs*i:bs*(i+1)].to(device)#.view(1, -1)
        outputs = net(inputs)
        y = torch.cat((y, outputs))

x = x[:len(x)-ext]
y = y[:len(y)-ext]

y = y.cpu().numpy()


print('Evaluation Finished!')

D:\ANACONDA\lib\site-packages\torch\nn\functional.py:1025: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


Evaluation Finished!


### Present the data in the CSV way and see if there are collisions

In [6]:
# separate the data per champs and extract which is the top, jng, mid, adc, supp
pos = y.reshape(-1, 5, 5).argmax(1)

# exhange elements with indices to see instead of [ who is assigned (top) ] , [ which role is assigned to (champion 1) ]
pos = np.argsort(pos, axis=1)

# check there were no collisions
exp = np.arange(5)
check = np.sort(pos, axis=1)
print("Free of collisions? "+str((check == exp).all()))

# this code will allow you to see how many and where the colllisions happened (ONLY IF THERE WERE COLLISIONS)
"""
count = 0
for e in range(len(pos)):
    if not (check[e] == exp).all():
        count += 1
print(str(count)+"/"+str(len(y)))
"""

print(x.reshape(-1, 5, 4)[100000])
print(pos[100000])

Free of collisions? True
tensor([[51,  4,  4,  5],
        [38, 13,  4,  1],
        [91,  0,  4,  0],
        [45,  3,  4,  7],
        [35,  6,  4,  9]])
[2 4 0 3 1]


### Check randomly and empirically the correctness

In [12]:
num_to_alpha = np.array(['Aatrox', 'Ahri', 'Akali', 'Alistar', 'Amumu', 'Anivia', 'Annie', 'Ashe', 'Aurelion Sol', 'Azir', 'Bard', 'Blitzcrank', 'Brand', 'Braum', 'Caitlyn', 'Camille', 'Cassiopeia',
                "Cho'Gath", 'Corki', 'Darius', 'Diana', 'Draven', 'Dr. Mundo', 'Ekko', 'Elise', 'Evelynn', 'Ezreal', 'Fiddlesticks', 'Fiora', 'Fizz', 'Galio', 'Gangplank', 'Garen', 'Gnar',
                'Gragas', 'Graves', 'Hecarim', 'Heimerdinger', 'Illaoi', 'Irelia', 'Ivern', 'Janna', 'Jarvan IV', 'Jax', 'Jayce', 'Jhin', 'Jinx', "Kai'Sa", 'Kalista', 'Karma', 'Karthus', 'Kassadin',
                'Katarina', 'Kayle', 'Kayn', 'Kennen', "Kha'Zix", 'Kindred', 'Kled', "Kog'Maw", 'LeBlanc', 'Lee Sin', 'Leona', 'Lissandra', 'Lucian', 'Lulu', 'Lux', 'Malphite', 'Malzahar',
                'Maokai', 'Master Yi', 'Miss Fortune', 'Wukong', 'Mordekaiser', 'Morgana', 'Nami', 'Nasus', 'Nautilus', 'Nidalee', 'Nocturne', 'Nunu', 'Olaf', 'Orianna', 'Ornn', 'Pantheon',
                'Poppy', 'Pyke', 'Quinn', 'Rakan', 'Rammus', "Rek'Sai", 'Renekton', 'Rengar', 'Riven', 'Rumble', 'Ryze', 'Sejuani', 'Shaco', 'Shen', 'Shyvana', 'Singed', 'Sion', 'Sivir', 'Skarner',
                'Sona', 'Soraka', 'Swain', 'Syndra', 'Tahm Kench', 'Taliyah', 'Talon', 'Taric', 'Teemo', 'Thresh', 'Tristana', 'Trundle', 'Tryndamere', 'Twisted Fate', 'Twitch', 'Udyr', 'Urgot',
                'Varus', 'Vayne', 'Veigar', "Vel'Koz", 'Vi', 'Viktor', 'Vladimir', 'Volibear', 'Warwick', 'Xayah', 'Xerath', 'Xin Zhao', 'Yasuo', 'Yorick', 'Zac', 'Zed', 'Ziggs', 'Zilean', 'Zoe', 'Zyra'])
spell_to_alpha = ['Teleport', 'Exhaust', 'Barrier', 'Clarity', 'Flash', 'Ignite', 'Ghost', 'Heal', 'Cleanse', 'Smite']
key_to_alpha = ['Press the Attack', 'Lethal Tempo', 'Conqueror', 'Fleet Footwork', 'Electrocute', 'Predator', 'Dark Harvest', 'Summon Aery', 'Arcane Comet', 'Phase Rush',
                'Glacial Augment', 'Kleptomancy', 'Unsealed Spellbook', 'Grasp of the Undying', 'Aftershock', 'Guardian', 'Hail of Blades']
pos_to_alpha = np.array(['TOP', 'JNG', 'MID', 'ADC', 'SUP'])

ind = np.random.randint(0, len(pos))

x_t = x.reshape(-1, 5, 4)

#for i in range(5):
#    print(num_to_alpha[x_t[ind][i][0]], pos_to_alpha[pos[ind][i]])

for i in range(5):
    cac = x_t[ind][np.where(pos[ind] == i)][0]
    print(pos_to_alpha[i]+": "+num_to_alpha[cac[0]]+" ("+key_to_alpha[cac[1]]
          +", "+spell_to_alpha[cac[2]]+", "+spell_to_alpha[cac[3]]+")")

TOP: Gangplank (Grasp of the Undying, Flash, Teleport)
JNG: Nidalee (Dark Harvest, Flash, Smite)
MID: Ryze (Phase Rush, Flash, Cleanse)
ADC: Ezreal (Kleptomancy, Flash, Teleport)
SUP: Karma (Summon Aery, Flash, Ignite)


### Save to the CSV in the specified format

In [14]:
# i.e. ENEMY, ROWID, CHAMP 1 POS, CHAMP 2 POS, CHAMP 3 POS, CHAMP 4 POS, CHAMP 5 POS

to_csv = np.concatenate((misc, pos), 1)
pd.DataFrame(to_csv).to_csv("correctedData.csv", index=False, 
                            header=["ENEMY", "ROWID", "CH1Pos", "CH2Pos", "CH3Pos", "CH4Pos", "CH5Pos"])